### Playerbase data dump

This notebook generates a raw .json file of playerbase information.

In [1]:
import requests
import json
import time
import random
import os

In [3]:
# Obtain API authorization keys
with open('keys.txt', 'r') as f:
    lines = f.readlines()
    client_id = lines[0].split('\"')[1]
    client_secret = lines[1].split('\"')[1]
    username = lines[2].split('\"')[1]
    password = lines[3].split('\"')[1]

# Obtaining authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={'grant_type':'password','username':username,'password':password,'client_id':client_id,'client_secret':client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

The following cell fetches information for the player's games. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [4]:
# Set endpoint, request first page of results
endpoint = 'https://online-go.com/api/v1/players/'+str(player_id)+'/games'
gamesPage = s.get(endpoint, headers = headers).json()
games = gamesPage['results']
nextPage = gamesPage['next']

attempts = 0
max_attempts = 10
while nextPage != None:
    response = s.get(nextPage,  headers = headers)
    # If not rate limited, continue
    if response.status_code != 429:
        attempts = 0
        print(nextPage)
        gamesPage = response.json()
        games.extend(gamesPage['results'])
        nextPage = gamesPage['next']
    # If rate limited, wait and then start again
    else:
        print('Rate limit reached. Backing off before retrying query. Please wait...')
        attempts += 1
        if attempts >= max_attempts:
            print('Too many attempts.')
            break
        else:
            time.sleep((2 ** attempts) + random.random())

https://online-go.com/api/v1/players/1024823/games?page=2
https://online-go.com/api/v1/players/1024823/games?page=3


The final cell creates a JSON file named `raw_stats.json` in the `./output/<player_id>/datasets` directory containing the raw data dump of the player's games.

In [5]:
try:
    os.mkdir('output')
except FileExistsError:
    pass
try:
    os.mkdir('output/'+str(player_id))
except FileExistsError:
    pass
try:
    os.mkdir('output/'+str(player_id)+'/datasets')
except FileExistsError:
    pass

with open('./output/'+str(player_id)+'/datasets/raw_stats.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(games, indent = 2))